In [1]:
from tqdm import tqdm 
import numpy as np

# Thermal Expectations

General idea of this notebook is to use Quantum Monte Carlo methods to sample thermal expecation values. 

Current plan 
- [ ] Implement general Metropolis-Hastings algorithm to calculate thermal expecations
- [ ] Add sampling through TN states
- [ ] Add straightforward QMC sampling of states
- [ ] Create code for experiments to track burn in 

## General Metropolis-Hasting algo

In [2]:
def metropolisThermalExpectation(O, initialState,
                                 proposalFunc,
                                 observableExpectation,
                                 boltzmannExpectation,
                                 T,
                                 warmup=100, steps=1000):
    '''
    Use the Metropolis Hastings algorithm to sample thermal expectation values. 

        Parameters
        ----------
        O : array
            Operator to calculate expectation. 
        proposalFunc : function 
            Generator function for sampling from propsal distribution of new states 
        observableExpectation : 
            Calculate the observable expectation weighted by the Boltzmann factor
        boltzmannExpectation : 
            Calculate the Boltzmann weight
        T : int
            Temperature to simulate
        iterations : 
            Number of steps of the chain to take 

        Returns 
        -------
        array : A list of outputs from the `observableExpectation`function
    '''
    k = 1 # Boltzmann constant 
    Os = np.zeros(steps) # Storage for observables
    currentState = deepcopy(initialState)

    # Warmup the MC
    print('Warming up MC...')
    for n in tqdm(range(warmup)): 
        currentState = metropolisStep1DIsing(currentState, proposalFunc) 

    # Run the MC + calculate observables at each step 
    print('Running MC...')
    for n in tqdm(range(steps)): 
        currentState = metropolisStep1DIsing(currentState, proposalFunc)
        Os[n] = observableExpectation(currentState, O)

    return Os

In [3]:
def metropolisStep1DIsing(oldState, sampleProposal): 
    # sample new state 
    newState = sampleProposal(oldState)

    # calculate Boltzmann weights ratio
    weightRatio = calculateBoltzmannWeightRatio(oldState, newState) 

    # calcualte acceptance 
    acceptanceRatio = min(1, weightRatio)
    if random.rand() < acceptanceRatio: 
        return newState
    return oldState

## Functions for working with MPS states

In [4]:
from scipy.stats import unitary_group

In [16]:
def randomUnitaryState(oldState):
    '''
    Generate a random left canonical state as a proposal state. 
    '''
    l, rho, r = oldState.shape
    D = max(l*rho, r)
    newState = unitary_group.rvs(D)
    newState = newState[:l*rho, :r]
    return newState.reshape(l, rho, r)

In [ ]:
def mpsBoltzmannRatio(stateA, stateB, expBH):
    '''
    Calculate the Boltzmann ratio to check if we accept the new state. 
    '''
    expA = expectation(stateA, expBH)
    expB = expectation(stateB, expBH)
    return expA/expB

In [ ]:
def expectation(state, O):
    '''
    Calculate translationally invariant expectation of operator O given `state`. 
    '''
    Dl, rho, Dr = state.shape 
    Odim = O.shape[0]
    nStates = Odim // rho
    Oten = O.reshape(2 * nStates * [rho])

    tensors = [ state

## References 
- Code: 
https://github.com/prtkm/ising-monte-carlo/blob/master/ising-monte-carlo.org
https://rajeshrinet.github.io/blog/2014/ising-model/

- Analytics:
https://stanford.edu/~jeffjar/statmech/lec3.html

- Manuscript
https://www.cond-mat.de/events/correl13/manuscripts/wessel.pdf = Uses Heisenberg model as reference but it might be a good test to reproduce some of the 1D behaviour we expect. 